In [1]:
import sys, os
sys.path.append("..")

import pandas as pd
import yfinance as yf

from agents import StrategyAgent, RiskAgent, AllocatorAgent


In [2]:
tickers = ["AAPL", "MSFT", "AMZN"]

data = {}
for ticker in tickers:
    df = yf.download(ticker, start="2020-01-01", end="2023-12-31", progress=False)
    df["Return"] = df["Close"].pct_change()
    data[ticker] = df

print("Loaded tickers:", list(data.keys()))


C:\Users\adity\AppData\Local\Temp\ipykernel_26532\3750658778.py:5: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, start="2020-01-01", end="2023-12-31", progress=False)
C:\Users\adity\AppData\Local\Temp\ipykernel_26532\3750658778.py:5: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, start="2020-01-01", end="2023-12-31", progress=False)
C:\Users\adity\AppData\Local\Temp\ipykernel_26532\3750658778.py:5: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, start="2020-01-01", end="2023-12-31", progress=False)


Loaded tickers: ['AAPL', 'MSFT', 'AMZN']


In [3]:
results = {}

for ticker, df in data.items():
    strat_agent = StrategyAgent(df)
    strategies = strat_agent.generate()   # assume this gives SMA/RSI/Bollinger
    
    ticker_results = {}
    risk_agent = RiskAgent()
    
    for name, strat_df in strategies.items():
        metrics = risk_agent.evaluate(strat_df["Strategy_Return"])
        ticker_results[name] = {
            "Metrics": metrics,
            "Data": strat_df
        }
    
    results[ticker] = ticker_results


In [4]:
combined = pd.DataFrame()

for ticker, strat_dict in results.items():
    for name, out in strat_dict.items():
        metrics = risk_agent.evaluate(out["Data"]["Strategy_Return"])

        if metrics["Sharpe"] > 1.0 and metrics["Decision"] == "PASS":
            combined[f"{ticker}_{name}"] = out["Data"]["Strategy_Return"]



In [5]:
os.makedirs("../logs", exist_ok=True)
os.makedirs("../data", exist_ok=True)

# Save combined portfolio
combined.to_csv("../logs/Day22_multi_ticker_portfolio.csv")

# Save metrics summary
summary = combined.describe().T
summary.to_csv("../logs/Day22_summary.csv")
